In [1]:
from google.colab import files
uploaded=files.upload()

Saving cities_coordinates.xlsx to cities_coordinates.xlsx


In [2]:
import pandas as pd
import numpy as np
from math import radians, sin, cos, sqrt, atan2
from scipy.spatial import distance


df=pd.read_excel('cities_coordinates.xlsx')
df.columns=df.columns.str.lower()
df[['latitude','longitude']]=df['coordinates'].str.split(', ',expand=True)
df[['latitude','longitude']]=df[['latitude','longitude']].astype(float)
df

,city,coordinates,latitude,longitude
0,Athens,"37.984584952303656, 23.72734833182475",37.984585,23.727348
1,Thessaloniki,"40.64020572394406, 22.946026116345354",40.640206,22.946026
2,Milano,"45.47032037264584, 9.185268599057604",45.470320,9.185269
3,Madrid,"40.42058886968387, -3.702214075001009",40.420589,-3.702214
4,Barcelona,"41.3873989954091, 2.1724435638695256",41.387399,2.172444
5,Porto,"41.15998089666878, -8.628501974538407",41.159981,-8.628502
6,London,"51.506815690955484, -0.13736314098574948",51.506816,-0.137363
7,Copenhagen,"55.6853354925537, 12.570183861730067",55.685335,12.570184
8,Stockholm,"59.33693358328568, 18.067148325018334",59.336934,18.067148
9,Bucuresti,"44.42005477711049, 26.105606835533933",44.420055,26.105607


In [3]:
# Function to calculate Haversine distance
def haversine_distance(coord1, coord2):
    R = 6371.0  # Radius of the Earth in kilometers
    lat1, lon1 = map(radians, coord1)
    lat2, lon2 = map(radians, coord2)

    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c

    return distance


# Function to calculate distances between all pairs of cities
def calculate_distances(df):
    distances = pd.DataFrame(index=df['city'], columns=df['city'])

    for city1 in df['city']:
        for city2 in df['city']:
            coord1 = (df.loc[df['city'] == city1, 'latitude'].values[0], df.loc[df['city'] == city1, 'longitude'].values[0])
            coord2 = (df.loc[df['city'] == city2, 'latitude'].values[0], df.loc[df['city'] == city2, 'longitude'].values[0])
            distances.loc[city1, city2] = haversine_distance(coord1, coord2)

    return distances

# Calculate distances and display the result
distance_matrix = calculate_distances(df)
print("Distance Matrix:")
print(distance_matrix)

Distance Matrix:
city               Athens Thessaloniki       Milano       Madrid    Barcelona  \
city                                                                            
Athens                0.0   302.841511  1462.438914  2369.381984  1877.683479   
Thessaloniki   302.841511          0.0  1238.153319  2243.632843  1740.755504   
Milano        1462.438914  1238.153319          0.0  1187.820348   725.347354   
Madrid        2369.381984  2243.632843  1187.820348          0.0   505.177395   
Barcelona     1877.683479  1740.755504   725.347354   505.177395          0.0   
Porto         2779.478229  2639.722155  1515.031179   422.734374   902.409665   
London        2392.100111  2137.313054   959.106532  1262.750185  1138.903293   
Copenhagen    2137.199581  1836.806143  1160.373571  2073.043094  1760.123836   
Stockholm     2408.332922  2106.671061  1652.049973   2593.13276  2279.001162   
Bucuresti      742.644061   493.547429  1334.284318  2472.650173  1970.977951   
Budapest   

In [ ]:
def nearest_neighbor(distance_matrix,start):
    num_cities = len(distance_matrix)
    unvisited_cities = set(distance_matrix.drop(start,axis=0).index)   # Cities are labeled from 0 to num_cities-1

    current_city = start  # Start from the first city
    tour = [current_city]

    while unvisited_cities:
        nearest_city = min(unvisited_cities, key=lambda city: distance_matrix[current_city][city])
        tour.append(nearest_city)
        unvisited_cities.remove(nearest_city)
        current_city = nearest_city

    # Return to the starting city to complete the tour
    tour.append(tour[0])

    return tour
nearest_neighbor(distance_matrix,'Athens')

['Athens',
 'Thessaloniki',
 'Bucuresti',
 'Belgrade',
 'Budapest',
 'Vienna',
 'Prague',
 'Berlin',
 'Copenhagen',
 'Stockholm',
 'London',
 'Paris',
 'Milano',
 'Rome',
 'Valetta',
 'Barcelona',
 'Madrid',
 'Porto',
 'Athens']

In [ ]:
def minimum_insertions(distance_matrix,start):
  num_cities = len(distance_matrix)
  unvisited_cities = set(distance_matrix.drop(start,axis=0).index)   # Cities are labeled from 0 to num_cities-1

  visited_city = start  # Start from the first city
  tour = [visited_city]

  while unvisited_cities:
      nearest_city=[];nearest_dist=[]
      for v_city in tour:
        nearest_city.append(min(unvisited_cities, key=lambda un_city: distance_matrix[v_city][un_city]))
        nearest_dist.append(distance_matrix[v_city][min(unvisited_cities, key=lambda un_city: distance_matrix[v_city][un_city])])

      idx=np.argsort(nearest_dist)[0]
      city=nearest_city[idx]
      tour.insert(idx+1,city)
      unvisited_cities.remove(city)

  tour.append(tour[0]) # Return to the starting city to complete the tour
  return tour
minimum_insertions(distance_matrix,start='Athens')

['Athens',
 'Thessaloniki',
 'Bucuresti',
 'Belgrade',
 'Budapest',
 'Vienna',
 'Milano',
 'Barcelona',
 'Madrid',
 'Porto',
 'Paris',
 'London',
 'Rome',
 'Valetta',
 'Prague',
 'Berlin',
 'Copenhagen',
 'Stockholm',
 'Athens']

In [4]:
class RouteOptimization:
  def __init__(self,distance_matrix,start):
     self.distance_matrix=distance_matrix
     self.start=start

  def nearest_neighbor(self):
    num_cities = len(self.distance_matrix)
    unvisited_cities = set(self.distance_matrix.drop(self.start,axis=0).index)   # Cities are labeled from 0 to num_cities-1

    current_city = self.start  # Start from the first city
    tour = [current_city]

    while unvisited_cities:
        nearest_city = min(unvisited_cities, key=lambda city: self.distance_matrix[current_city][city])
        tour.append(nearest_city)
        unvisited_cities.remove(nearest_city)
        current_city = nearest_city

    # Return to the starting city to complete the tour
    tour.append(tour[0])

    return tour

  def minimum_insertions(self):
    num_cities = len(self.distance_matrix)
    unvisited_cities = set(self.distance_matrix.drop(self.start,axis=0).index)   # Cities are labeled from 0 to num_cities-1

    visited_city = self.start  # Start from the first city
    tour = [visited_city]

    while unvisited_cities:
        nearest_city=[];nearest_dist=[]
        for v_city in tour:
          nearest_city.append(min(unvisited_cities, key=lambda un_city: self.distance_matrix[v_city][un_city]))
          nearest_dist.append(self.distance_matrix[v_city][min(unvisited_cities, key=lambda un_city: self.distance_matrix[v_city][un_city])])

        idx=np.argsort(nearest_dist)[0]
        city=nearest_city[idx]
        tour.insert(idx+1,city)
        unvisited_cities.remove(city)

    tour.append(tour[0]) # Return to the starting city to complete the tour
    return tour


In [5]:
fit=RouteOptimization(distance_matrix,'Athens')
fit.minimum_insertions()

['Athens',
 'Thessaloniki',
 'Bucuresti',
 'Belgrade',
 'Budapest',
 'Vienna',
 'Milano',
 'Barcelona',
 'Madrid',
 'Porto',
 'Paris',
 'London',
 'Prague',
 'Berlin',
 'Copenhagen',
 'Stockholm',
 'Athens']

In [ ]:
data = [0, 52, 37, 46, 47, 28, 52, 0, 84, 90, 97, 41, 37, 84, 0, 11, 15, 47, 46, 90, 11, 0, 20, 50, 47, 97, 15, 20, 0, 61, 28, 41, 47, 50, 61, 0]

# Create a 5x5 data frame
df1 = pd.DataFrame(np.array(data).reshape(6, 6), index=range(0, 6), columns=range(0, 6))
df1.index = df1.index.astype(str)
df1.columns = df1.columns.astype(str)
# Print the data frame
print(df1)

    0   1   2   3   4   5
0   0  52  37  46  47  28
1  52   0  84  90  97  41
2  37  84   0  11  15  47
3  46  90  11   0  20  50
4  47  97  15  20   0  61
5  28  41  47  50  61   0


In [ ]:

num_cities = len(df1)
unvisited_cities = set(df1.index[1:])   # Cities are labeled from 0 to num_cities-1

current_city = '0'  # Start from the first city
tour = [current_city]

while unvisited_cities:
    nearest_city = min(unvisited_cities, key=lambda city: df1[current_city][city])
    tour.append(nearest_city)
    unvisited_cities.remove(nearest_city)
    current_city = nearest_city

# Return to the starting city to complete the tour
tour.append(tour[0])

tour

NameError: ignored

In [ ]:
num_cities = len(df1)
unvisited_cities = set(df1.index[1:])   # Cities are labeled from 0 to num_cities-1

visited_city = '0'  # Start from the first city
tour = [visited_city]

while unvisited_cities:
    nearest_city=[];nearest_dist=[]
    for v_city in tour:
      nearest_city.append(min(unvisited_cities, key=lambda un_city: df1[v_city][un_city]))
      nearest_dist.append(df1[v_city][min(unvisited_cities, key=lambda un_city: df1[v_city][un_city])])

    idx=np.argsort(nearest_dist)[0]
    city=nearest_city[idx]
    tour.insert(idx+1,city)
    unvisited_cities.remove(city)

# Return to the starting city to complete the tour
tour.append(tour[0])
tour

['0', '2', '4', '3', '5', '1', '0']

In [ ]:
minimum_insertions(df1,'0')

['0', '2', '4', '3', '5', '1', '0']

In [7]:
#calculate the cost for each route
# make the formatting for Tableau
x=fit.minimum_insertions()
range(len(x))

range(0, 17)

In [8]:
path_data=pd.DataFrame()
for i in range(len(x)-1):
  origin=pd.DataFrame({'origin_city':[x[i]],
      'origin_latitude':[df[df['city']==x[i]]['latitude'].values[0]],
      'origin_longitude':[df[df['city']==x[i]]['longitude'].values[0]]})
  destination=pd.DataFrame({'destination_city':[x[i+1]],
      'destination_latitude':[df[df['city']==x[i+1]]['latitude'].values[0]],
      'destination_longitude':[df[df['city']==x[i+1]]['longitude'].values[0]],
      'path':[i+1]})

  axis1=pd.concat([origin,destination],axis=1)
  path_data=pd.concat([path_data,axis1],axis=0)
path_data.reset_index(drop=True,inplace=True)

In [9]:
path_data

,origin_city,origin_latitude,origin_longitude,destination_city,destination_latitude,destination_longitude,path
0,Athens,37.984585,23.727348,Thessaloniki,40.640206,22.946026,1
1,Thessaloniki,40.640206,22.946026,Bucuresti,44.420055,26.105607,2
2,Bucuresti,44.420055,26.105607,Belgrade,44.814153,20.463779,3
3,Belgrade,44.814153,20.463779,Budapest,47.504867,19.053335,4
4,Budapest,47.504867,19.053335,Vienna,48.216854,16.354953,5
5,Vienna,48.216854,16.354953,Milano,45.470320,9.185269,6
6,Milano,45.470320,9.185269,Barcelona,41.387399,2.172444,7
7,Barcelona,41.387399,2.172444,Madrid,40.420589,-3.702214,8
8,Madrid,40.420589,-3.702214,Porto,41.159981,-8.628502,9
9,Porto,41.159981,-8.628502,Paris,48.853404,2.352524,10
